In [33]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd   
import re
import os   
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing


In [2]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd "/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity"

/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity


In [ ]:
data = pd.read_csv("labeled_data.csv")

In [ ]:
data.drop(columns=["Unnamed: 0","text_1","text_2"],inplace=True)

In [ ]:
data.head()

,label,POS dist score,1-gram_overlap,chrf_score_norm,WMD,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,BertScore,L2_score
0,4.000,5.000000,3.333333,2.533040,1.382194,3.000000,3.750000,3.333333,0.0,0.0,0.0,3.000000,3.750000,3.333333,4.5364,2.621138
1,3.000,0.000000,1.666667,0.869565,3.197893,1.666667,1.666667,1.666667,0.0,0.0,0.0,1.666667,1.666667,1.666667,4.5531,2.147480
2,2.333,5.000000,3.333333,2.658046,2.067205,3.333333,3.333333,3.333333,0.0,0.0,0.0,3.333333,3.333333,3.333333,4.7772,2.539115
3,2.667,2.305241,0.000000,2.475248,2.630209,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.3771,0.842952
4,3.000,2.688024,2.500000,1.500000,2.365262,2.500000,1.666667,2.000000,0.0,0.0,0.0,2.500000,1.666667,2.000000,4.4446,1.865555


In [ ]:
X = data.drop(["label"], axis=1).copy()
X.shape

(998, 15)

In [ ]:
X.head()

,POS dist score,1-gram_overlap,chrf_score_norm,WMD,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,BertScore,L2_score
0,5.000000,3.333333,2.533040,1.382194,3.000000,3.750000,3.333333,0.0,0.0,0.0,3.000000,3.750000,3.333333,4.5364,2.621138
1,0.000000,1.666667,0.869565,3.197893,1.666667,1.666667,1.666667,0.0,0.0,0.0,1.666667,1.666667,1.666667,4.5531,2.147480
2,5.000000,3.333333,2.658046,2.067205,3.333333,3.333333,3.333333,0.0,0.0,0.0,3.333333,3.333333,3.333333,4.7772,2.539115
3,2.305241,0.000000,2.475248,2.630209,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.3771,0.842952
4,2.688024,2.500000,1.500000,2.365262,2.500000,1.666667,2.000000,0.0,0.0,0.0,2.500000,1.666667,2.000000,4.4446,1.865555


In [ ]:
y = data["label"].copy()
y.head()

0    4.000
1    3.000
2    2.333
3    2.667
4    3.000
Name: label, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = RandomForestRegressor(max_depth=2)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=2, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
# Use the forest's predict method on the test data
y_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

0.5704996566016303

In [ ]:
feature_importance_dict = {'feature': X.columns.values, 'importance':model.feature_importances_} 

In [ ]:
feature_importance_df = pd.DataFrame(feature_importance_dict).sort_values('importance', ascending=False) 

In [ ]:
feature_importance_df

,feature,importance
3,WMD,0.376846
2,chrf_score_norm,0.248155
13,BertScore,0.195598
14,L2_score,0.087413
5,ROUGE-1 precision,0.021447
12,ROUGE-L F,0.021338
11,ROUGE-L precision,0.017681
1,1-gram_overlap,0.010945
7,ROUGE-2 recall,0.009196
10,ROUGE-L recall,0.004518


In [ ]:
all_y_pred = model.predict(X)

In [ ]:
mean_squared_error(y, all_y_pred)

0.5450433622192488

In [ ]:
new_data = data.copy() 
new_data['Predictions'] = all_y_pred

In [ ]:
new_data.head()

,label,POS dist score,1-gram_overlap,chrf_score_norm,WMD,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,BertScore,L2_score,Predictions
0,4.000,5.000000,3.333333,2.533040,1.382194,3.000000,3.750000,3.333333,0.0,0.0,0.0,3.000000,3.750000,3.333333,4.5364,2.621138,3.932881
1,3.000,0.000000,1.666667,0.869565,3.197893,1.666667,1.666667,1.666667,0.0,0.0,0.0,1.666667,1.666667,1.666667,4.5531,2.147480,3.390540
2,2.333,5.000000,3.333333,2.658046,2.067205,3.333333,3.333333,3.333333,0.0,0.0,0.0,3.333333,3.333333,3.333333,4.7772,2.539115,3.716750
3,2.667,2.305241,0.000000,2.475248,2.630209,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.3771,0.842952,3.447618
4,3.000,2.688024,2.500000,1.500000,2.365262,2.500000,1.666667,2.000000,0.0,0.0,0.0,2.500000,1.666667,2.000000,4.4446,1.865555,3.367112


In [ ]:
new_data.to_csv('Paraphrase_labeled_data_with_predictions.csv', index=False)
!cp Paraphrase_labeled_data_with_predictions.csv "/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity"

cp: 'Paraphrase_labeled_data_with_predictions.csv' and '/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity/Paraphrase_labeled_data_with_predictions.csv' are the same file


# run the model on the full dataset

In [4]:
combined_data = pd.read_pickle("./combined_data.pickle")

In [5]:
combined_data.head()

,dataset,label,text_1,text_2,bleu_allwords,bleu_withoutstop,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop,WMD,1-gram_overlap,2-gram_overlap,3-gram_overlap,4-gram_overlap,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,chrf_score,chrf_score_norm,POS dist score,text_1_tokens,text_2_tokens,L2_score,bert
0,2012.MSRpar.test.tsv,4.4,The problem likely will mean corrective change...,He said the problem needs to be corrected befo...,0.375739,0.333333,96.20,90.82,77.23,77.39,3,0.400000,0.133333,0.000000,0.000000,0.466667,0.368421,0.411765,0.214286,0.166667,0.187500,0.466667,0.368421,0.411765,0.536815,2.684077,3.055075,"[The, problem, likely, will, mean, corrective,...","[He, said, the, problem, needs, to, be, correc...",10.527886,0.926813
1,2012.MSRpar.test.tsv,0.8,The technology-laced Nasdaq Composite Index .I...,The broad Standard & Poor 's 500 Index .SPX in...,0.461538,0.333333,95.66,88.96,87.95,72.39,3,0.400000,0.117647,0.000000,0.000000,0.500000,0.428571,0.461538,0.176471,0.150000,0.162162,0.500000,0.428571,0.461538,0.391649,1.958247,3.042741,"[The, technology, laced, Nasdaq, Composite, In...","[The, broad, Standard, Poor, s, 500, Index, SP...",11.930013,0.891672
2,2012.MSRpar.test.tsv,3.6,"`` It 's a huge black eye , '' said publisher ...","`` It 's a huge black eye , '' Arthur Sulzberg...",0.578947,0.500000,97.75,94.94,88.39,72.44,2,0.647059,0.294118,0.250000,0.200000,0.640000,0.727273,0.680851,0.333333,0.380952,0.355556,0.560000,0.636364,0.595745,0.549542,2.747710,4.329347,"[It, s, a, huge, black, eye, said, publisher, ...","[It, s, a, huge, black, eye, Arthur, Sulzberge...",9.865285,0.937471
3,2012.MSRpar.test.tsv,3.4,SEC Chairman William Donaldson said there is a...,`` I think there 's a building confidence that...,0.578947,0.444444,98.17,90.51,94.16,75.97,2,0.689655,0.516129,0.413793,0.296296,0.636364,0.823529,0.717949,0.428571,0.562500,0.486486,0.590909,0.764706,0.666667,0.547146,2.735730,0.000000,"[SEC, Chairman, William, Donaldson, said, ther...","[I, think, there, s, a, building, confidence, ...",10.122802,0.923259
4,2012.MSRpar.test.tsv,1.4,Vivendi shares closed 1.9 percent at 15.80 eur...,"In New York , Vivendi shares were 1.4 percent ...",0.384615,0.375000,95.77,87.00,90.74,67.48,3,0.322581,0.066667,0.000000,0.000000,0.294118,0.357143,0.322581,0.062500,0.076923,0.068966,0.294118,0.357143,0.322581,0.293020,1.465099,0.000000,"[Vivendi, shares, closed, 1, 9, percent, at, 1...","[In, New, York, Vivendi, shares, were, 1, 4, p...",14.036093,0.908997


In [18]:
combined_data.isnull().values.any()
combined_data.isnull().sum().sum()

1124

In [20]:
combined_data.shape

(25767, 31)

In [22]:
combined_data.dropna(inplace= True)
combined_data.shape

(24902, 31)

In [26]:
combined_data.to_pickle('combined_data_no_nans.pickle')
!cp combined_data_no_nans.pickle "/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity"

cp: 'combined_data_no_nans.pickle' and '/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity/combined_data_no_nans.pickle' are the same file


In [61]:
X = combined_data.drop(["dataset", "label","text_1", "text_2", "text_1_tokens","text_2_tokens"  ], axis=1).copy()
X.head()

,bleu_allwords,bleu_withoutstop,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop,WMD,1-gram_overlap,2-gram_overlap,3-gram_overlap,4-gram_overlap,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,chrf_score,chrf_score_norm,POS dist score,L2_score,bert
0,0.375739,0.333333,96.20,90.82,77.23,77.39,3,0.400000,0.133333,0.000000,0.000000,0.466667,0.368421,0.411765,0.214286,0.166667,0.187500,0.466667,0.368421,0.411765,0.536815,2.684077,3.055075,10.527886,0.926813
1,0.461538,0.333333,95.66,88.96,87.95,72.39,3,0.400000,0.117647,0.000000,0.000000,0.500000,0.428571,0.461538,0.176471,0.150000,0.162162,0.500000,0.428571,0.461538,0.391649,1.958247,3.042741,11.930013,0.891672
2,0.578947,0.500000,97.75,94.94,88.39,72.44,2,0.647059,0.294118,0.250000,0.200000,0.640000,0.727273,0.680851,0.333333,0.380952,0.355556,0.560000,0.636364,0.595745,0.549542,2.747710,4.329347,9.865285,0.937471
3,0.578947,0.444444,98.17,90.51,94.16,75.97,2,0.689655,0.516129,0.413793,0.296296,0.636364,0.823529,0.717949,0.428571,0.562500,0.486486,0.590909,0.764706,0.666667,0.547146,2.735730,0.000000,10.122802,0.923259
4,0.384615,0.375000,95.77,87.00,90.74,67.48,3,0.322581,0.066667,0.000000,0.000000,0.294118,0.357143,0.322581,0.062500,0.076923,0.068966,0.294118,0.357143,0.322581,0.293020,1.465099,0.000000,14.036093,0.908997


In [62]:
column_names = list(X.columns) 
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 5))
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled, columns=column_names)
X.head()

,bleu_allwords,bleu_withoutstop,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop,WMD,1-gram_overlap,2-gram_overlap,3-gram_overlap,4-gram_overlap,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,chrf_score,chrf_score_norm,POS dist score,L2_score,bert
0,1.878693,1.666667,4.831829,4.618232,3.758451,3.941083,1.875,2.000000,0.666667,0.000000,0.000000,2.333333,1.842105,2.058824,1.071429,0.833333,0.937500,2.333333,1.842105,2.058824,2.632840,2.632840,0.193343,1.331841,3.520558
1,2.307692,1.666667,4.807931,4.540880,4.342966,3.706913,1.875,2.000000,0.588235,0.000000,0.000000,2.500000,2.142857,2.307692,0.882353,0.750000,0.810811,2.500000,2.142857,2.307692,1.890952,1.890952,0.192755,1.510384,2.810210
2,2.894737,2.500000,4.900425,4.789570,4.366957,3.709254,1.250,3.235294,1.470588,1.250000,1.000000,3.200000,3.636364,3.404255,1.666667,1.904762,1.777778,2.800000,3.181818,2.978723,2.697881,2.697881,0.254100,1.247467,3.736010
3,2.894737,2.222222,4.919012,4.605340,4.681570,3.874578,1.250,3.448276,2.580645,2.068966,1.481481,3.181818,4.117647,3.589744,2.142857,2.812500,2.432432,2.954545,3.823529,3.333333,2.685636,2.685636,0.047679,1.280259,3.448716
4,1.923077,1.875000,4.812799,4.459370,4.495093,3.476958,1.875,1.612903,0.333333,0.000000,0.000000,1.470588,1.785714,1.612903,0.312500,0.384615,0.344828,1.470588,1.785714,1.612903,1.386893,1.386893,0.047679,1.778567,3.160412


In [66]:
y = combined_data["label"].copy()
y.head()

0    4.4
1    0.8
2    3.6
3    3.4
4    1.4
Name: label, dtype: float64

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [68]:
model = RandomForestRegressor(max_depth=2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred)

1.331753582849043

In [69]:
feature_importance_dict = {'feature': X.columns.values, 'importance':model.feature_importances_} 
feature_importance_df = pd.DataFrame(feature_importance_dict).sort_values('importance', ascending=False) 
feature_importance_df

,feature,importance
1,bleu_withoutstop,0.905143
5,ftext_withoutstop,0.094857
0,bleu_allwords,0.000000
14,ROUGE-2 recall,0.000000
23,L2_score,0.000000
22,POS dist score,0.000000
21,chrf_score_norm,0.000000
20,chrf_score,0.000000
19,ROUGE-L F,0.000000
18,ROUGE-L precision,0.000000


In [70]:
all_y_pred = model.predict(X)
mean_squared_error(y, all_y_pred)

1.318427074984877

In [71]:
new_data = combined_data.copy() 
new_data['Predictions'] = all_y_pred
new_data.head()

,dataset,label,text_1,text_2,bleu_allwords,bleu_withoutstop,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop,WMD,1-gram_overlap,2-gram_overlap,3-gram_overlap,4-gram_overlap,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,chrf_score,chrf_score_norm,POS dist score,text_1_tokens,text_2_tokens,L2_score,bert,Predictions
0,2012.MSRpar.test.tsv,4.4,The problem likely will mean corrective change...,He said the problem needs to be corrected befo...,0.375739,0.333333,96.20,90.82,77.23,77.39,3,0.400000,0.133333,0.000000,0.000000,0.466667,0.368421,0.411765,0.214286,0.166667,0.187500,0.466667,0.368421,0.411765,0.536815,2.684077,3.055075,"[The, problem, likely, will, mean, corrective,...","[He, said, the, problem, needs, to, be, correc...",10.527886,0.926813,2.514165
1,2012.MSRpar.test.tsv,0.8,The technology-laced Nasdaq Composite Index .I...,The broad Standard & Poor 's 500 Index .SPX in...,0.461538,0.333333,95.66,88.96,87.95,72.39,3,0.400000,0.117647,0.000000,0.000000,0.500000,0.428571,0.461538,0.176471,0.150000,0.162162,0.500000,0.428571,0.461538,0.391649,1.958247,3.042741,"[The, technology, laced, Nasdaq, Composite, In...","[The, broad, Standard, Poor, s, 500, Index, SP...",11.930013,0.891672,2.514165
2,2012.MSRpar.test.tsv,3.6,"`` It 's a huge black eye , '' said publisher ...","`` It 's a huge black eye , '' Arthur Sulzberg...",0.578947,0.500000,97.75,94.94,88.39,72.44,2,0.647059,0.294118,0.250000,0.200000,0.640000,0.727273,0.680851,0.333333,0.380952,0.355556,0.560000,0.636364,0.595745,0.549542,2.747710,4.329347,"[It, s, a, huge, black, eye, said, publisher, ...","[It, s, a, huge, black, eye, Arthur, Sulzberge...",9.865285,0.937471,3.074263
3,2012.MSRpar.test.tsv,3.4,SEC Chairman William Donaldson said there is a...,`` I think there 's a building confidence that...,0.578947,0.444444,98.17,90.51,94.16,75.97,2,0.689655,0.516129,0.413793,0.296296,0.636364,0.823529,0.717949,0.428571,0.562500,0.486486,0.590909,0.764706,0.666667,0.547146,2.735730,0.000000,"[SEC, Chairman, William, Donaldson, said, ther...","[I, think, there, s, a, building, confidence, ...",10.122802,0.923259,3.074263
4,2012.MSRpar.test.tsv,1.4,Vivendi shares closed 1.9 percent at 15.80 eur...,"In New York , Vivendi shares were 1.4 percent ...",0.384615,0.375000,95.77,87.00,90.74,67.48,3,0.322581,0.066667,0.000000,0.000000,0.294118,0.357143,0.322581,0.062500,0.076923,0.068966,0.294118,0.357143,0.322581,0.293020,1.465099,0.000000,"[Vivendi, shares, closed, 1, 9, percent, at, 1...","[In, New, York, Vivendi, shares, were, 1, 4, p...",14.036093,0.908997,2.861288


In [72]:
new_data.to_csv('combined_data_with_predictions.csv', index=False)
!cp combined_data_with_predictions.csv "/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity"

cp: 'combined_data_with_predictions.csv' and '/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity/combined_data_with_predictions.csv' are the same file


#run it on each dataset separately

In [74]:
datasets = combined_data.dataset.unique().tolist()
datasets

['2012.MSRpar.test.tsv',
 '2012.OnWN.test.tsv',
 '2012.SMTeuroparl.test.tsv',
 '2012.SMTnews.test.tsv',
 '2013.FNWN.test.tsv',
 '2013.headlines.test.tsv',
 '2013.OnWN.test.tsv',
 '2014.deft-forum.test.tsv',
 '2014.deft-news.test.tsv',
 '2014.headlines.test.tsv',
 '2014.images.test.tsv',
 '2014.OnWN.test.tsv',
 '2014.tweet-news.test.tsv',
 '2015.answers-forums.test.tsv',
 '2015.answers-students.test.tsv',
 '2015.belief.test.tsv',
 '2015.headlines.test.tsv',
 '2015.images.test.tsv',
 '2015.test.tsv',
 '2016.answer-answer.test.tsv',
 '2016.headlines.test.tsv',
 '2016.plagiarism.test.tsv',
 '2016.postediting.test.tsv',
 '2016.question-question.test.tsv',
 'Bible',
 'Bible_Random',
 'Paralex',
 'Paralex_Random',
 'Paraphrase',
 'Paraphrase_Random',
 'Yelp',
 'Yelp_Random']

In [76]:
frames = []
for dataset in datasets:
  print('********************************************************************')
  print('prediect labels for dataset '+dataset)
  dataset_df = combined_data.loc[combined_data['dataset'] == dataset]
  X = dataset_df.drop(["dataset", "label","text_1", "text_2", "text_1_tokens","text_2_tokens"  ], axis=1).copy()
  
  column_names = list(X.columns) 
  x = X.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 5))
  x_scaled = min_max_scaler.fit_transform(x)
  X = pd.DataFrame(x_scaled, columns=column_names)

  y = dataset_df["label"].copy()
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  model = RandomForestRegressor(max_depth=2)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print('mse: '+ str(mean_squared_error(y_test, y_pred)))

  feature_importance_dict = {'feature': X.columns.values, 'importance':model.feature_importances_} 
  feature_importance_df = pd.DataFrame(feature_importance_dict).sort_values('importance', ascending=False) 
  print('feature importance:')
  print(feature_importance_df)

  all_y_pred = model.predict(X)

  new_data = dataset_df.copy() 
  new_data['Predictions'] = all_y_pred

  frames.append(new_data)




********************************************************************
prediect labels for dataset 2012.MSRpar.test.tsv
mse: 0.5997687714190868
feature importance:
              feature  importance
7      1-gram_overlap    0.481505
24               bert    0.302109
21    chrf_score_norm    0.067454
20         chrf_score    0.056222
13          ROUGE-1 F    0.018653
1    bleu_withoutstop    0.013520
6                 WMD    0.013252
3   glove_withoutstop    0.013166
0       bleu_allwords    0.009865
11     ROUGE-1 recall    0.008547
23           L2_score    0.008139
2      glove_allwords    0.006079
5   ftext_withoutstop    0.001488
9      3-gram_overlap    0.000000
10     4-gram_overlap    0.000000
8      2-gram_overlap    0.000000
14     ROUGE-2 recall    0.000000
15  ROUGE-2 precision    0.000000
16          ROUGE-2 F    0.000000
17     ROUGE-L recall    0.000000
18  ROUGE-L precision    0.000000
19          ROUGE-L F    0.000000
4      ftext_allwords    0.000000
22     POS dist score 

In [77]:
full_prediction_result = pd.concat(frames)

In [78]:
full_prediction_result.head()

,dataset,label,text_1,text_2,bleu_allwords,bleu_withoutstop,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop,WMD,1-gram_overlap,2-gram_overlap,3-gram_overlap,4-gram_overlap,ROUGE-1 recall,ROUGE-1 precision,ROUGE-1 F,ROUGE-2 recall,ROUGE-2 precision,ROUGE-2 F,ROUGE-L recall,ROUGE-L precision,ROUGE-L F,chrf_score,chrf_score_norm,POS dist score,text_1_tokens,text_2_tokens,L2_score,bert,Predictions
0,2012.MSRpar.test.tsv,4.4,The problem likely will mean corrective change...,He said the problem needs to be corrected befo...,0.375739,0.333333,96.20,90.82,77.23,77.39,3,0.400000,0.133333,0.000000,0.000000,0.466667,0.368421,0.411765,0.214286,0.166667,0.187500,0.466667,0.368421,0.411765,0.536815,2.684077,3.055075,"[The, problem, likely, will, mean, corrective,...","[He, said, the, problem, needs, to, be, correc...",10.527886,0.926813,2.988756
1,2012.MSRpar.test.tsv,0.8,The technology-laced Nasdaq Composite Index .I...,The broad Standard & Poor 's 500 Index .SPX in...,0.461538,0.333333,95.66,88.96,87.95,72.39,3,0.400000,0.117647,0.000000,0.000000,0.500000,0.428571,0.461538,0.176471,0.150000,0.162162,0.500000,0.428571,0.461538,0.391649,1.958247,3.042741,"[The, technology, laced, Nasdaq, Composite, In...","[The, broad, Standard, Poor, s, 500, Index, SP...",11.930013,0.891672,2.078650
2,2012.MSRpar.test.tsv,3.6,"`` It 's a huge black eye , '' said publisher ...","`` It 's a huge black eye , '' Arthur Sulzberg...",0.578947,0.500000,97.75,94.94,88.39,72.44,2,0.647059,0.294118,0.250000,0.200000,0.640000,0.727273,0.680851,0.333333,0.380952,0.355556,0.560000,0.636364,0.595745,0.549542,2.747710,4.329347,"[It, s, a, huge, black, eye, said, publisher, ...","[It, s, a, huge, black, eye, Arthur, Sulzberge...",9.865285,0.937471,3.529694
3,2012.MSRpar.test.tsv,3.4,SEC Chairman William Donaldson said there is a...,`` I think there 's a building confidence that...,0.578947,0.444444,98.17,90.51,94.16,75.97,2,0.689655,0.516129,0.413793,0.296296,0.636364,0.823529,0.717949,0.428571,0.562500,0.486486,0.590909,0.764706,0.666667,0.547146,2.735730,0.000000,"[SEC, Chairman, William, Donaldson, said, ther...","[I, think, there, s, a, building, confidence, ...",10.122802,0.923259,3.426638
4,2012.MSRpar.test.tsv,1.4,Vivendi shares closed 1.9 percent at 15.80 eur...,"In New York , Vivendi shares were 1.4 percent ...",0.384615,0.375000,95.77,87.00,90.74,67.48,3,0.322581,0.066667,0.000000,0.000000,0.294118,0.357143,0.322581,0.062500,0.076923,0.068966,0.294118,0.357143,0.322581,0.293020,1.465099,0.000000,"[Vivendi, shares, closed, 1, 9, percent, at, 1...","[In, New, York, Vivendi, shares, were, 1, 4, p...",14.036093,0.908997,1.818037


In [79]:
full_prediction_result.shape

(24902, 32)

In [80]:
full_prediction_result.to_csv('combined_data_with_predictions_on_separate_datasets.csv', index=False)
!cp combined_data_with_predictions_on_separate_datasets.csv "/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity"

cp: 'combined_data_with_predictions_on_separate_datasets.csv' and '/content/drive/My Drive/Data Science Course/semester 2/NLP/homework/HW3 - semantic similarity/combined_data_with_predictions_on_separate_datasets.csv' are the same file
